In [1]:
import os
import cv2
import math
import random
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torch.autograd import Variable
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report
from torch import optim
import time

In [2]:
# parameters setting
input_shape = [3,96,96]
num_classes = 7
num_layers = 5
cut_size = 225
init_lr = 0.0003
batch_size = 128
use_fer=True
device= torch.device("cuda")

In [3]:
# load data
data_dir = ''

if use_fer:
    full_data = torch.load(data_dir + 'pickles/fer2013.pt')
    full_img, full_emo = full_data['images'][:,None,:,:], full_data['label']

    train_img,train_y = full_img[:28709],full_emo[:28709]
    test_img,test_y = full_img[28709:28709+3589],full_emo[28709:28709+3589]

    train_img = torch.cat((train_img,train_img,train_img),1)
    test_img = torch.cat((test_img,test_img,test_img),1)

else:
    train_data = torch.load(data_dir + 'pickles/AffectNet_train.pt')
    train_img, train_y = train_data['images'], train_data['emotion']
    
    test_data = torch.load(data_dir + 'pickles/AffectNet_test.pt')
    test_img, test_y = test_data['images'], test_data['emotion']
    

In [4]:
from torch.utils.data import Dataset, DataLoader
#======= data loaders =======#    

train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([int(cut_size*1.1),int(cut_size*1.1)]),
    transforms.RandomCrop([cut_size,cut_size]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([int(cut_size*1.1),int(cut_size*1.1)]),
    transforms.CenterCrop([cut_size,cut_size]),
    transforms.ToTensor()
])

def default_loader(images):
    img_tensor = train_transform(images)
    return img_tensor

def default_test_loader(images):
    img_tensor = test_transform(images)
    return img_tensor

class Dataset(Dataset):
    def __init__(self,imgs,ys,loader=default_loader):
        self.images = imgs 
        self.label = ys
        self.loader = loader

    def __getitem__(self, index):

        fn = self.images[index]
        img = self.loader(fn)
        label = self.label[index]
        return img,label

    
    def __len__(self):
        return len(self.images)

In [7]:
class AlexNet(nn.Module):

    def __init__(self,lr=0.0001,num_classes=7):
        super(AlexNet, self).__init__()
        self.model = models.alexnet(pretrained=True)
        self.model.classifier[1] = nn.Linear(9216, 4096)
        self.model.classifier[-1] = nn.Linear(4096,7)
        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer=torch.optim.Adam([p for p in self.parameters() if p.requires_grad] , lr=lr)
        

    def train_model(self, data_loader, num_epochs, lr):
        self.train()
        start=time.time()

        for epoch in range(num_epochs):
            running_loss=0
            running_acc = 0.
            total = 0
            num_batches = 0
            for count, (images, label) in enumerate(data_loader):
                self.optimizer.zero_grad()

                scores = 0.
                
                images=images.to(device)

                x = self.model.features(images).view(images.size(0),-1)
                scores = self.model.classifier(x)

                label=label.to(device)
                loss =  self.loss_fn(scores, label) 
                loss.backward()

                self.optimizer.step()

                running_loss += loss.detach().item()

                acc = (scores.argmax(-1) == label).float().sum()
                running_acc += acc.item()

                total+=images.size(0)
                num_batches += 1
                
            total_loss = running_loss/num_batches
            total_acc = running_acc/total
            elapsed = time.time()-start
            print('\t', 'epoch=',epoch, '\t time = {:.0f}m {:.0f}s'.format(elapsed // 60, elapsed % 60),'\t lr=', lr,'\t loss = ', total_loss , '\t Acc = {:.6f}'.format(total_acc*100),'%')

    def test_model(self, data_loader):
        self.eval()
        start=time.time()
        
        running_loss = 0.
        running_acc = 0
        total = 0
        num_batches = 0
        
        preds = []
        truth = []
        
        for count, (images, label) in enumerate(data_loader):
            self.optimizer.zero_grad()
            
            images = images.to(device)

            x = self.model.features(images).view(images.size(0),-1)
            scores = self.model.classifier(x).detach().cpu()

            acc = (scores.argmax(-1) == label).float().sum()
            running_acc += acc.item()
            
            total += images.size(0)

            preds.extend(scores.argmax(-1).numpy())
            truth.extend(label.numpy())
            
        total_acc = running_acc/total
        elapsed = time.time()-start

        print('testing','\t time = {:.0f}m {:.0f}s'.format(elapsed // 60, elapsed % 60), '\n Acc = {:.6f}'.format(total_acc*100),'%')
        test_fscore = classification_report(truth, preds)
        print(test_fscore)

In [ ]:
train_input  = Dataset(train_img,train_y)
train_loader = DataLoader(train_input, batch_size=batch_size,shuffle=True)

test_input  = Dataset(test_img,test_y,default_test_loader)
test_loader = DataLoader(test_input, batch_size=batch_size,shuffle=False)

net = AlexNet(lr=init_lr)
net.to(device)
print(net)
init_lr=0.0003
for i in range(20):
    print('Iteration', i)
    net.train_model(data_loader=train_loader, num_epochs=5,lr=init_lr)
    preds=net.test_model(data_loader=test_loader)